<a href="https://colab.research.google.com/github/priyvantkumar9661-png/Disease-Predictor-/blob/main/Disease_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d redwankarimsony/heart-disease-data -p /content/heart-disease --unzip

In [ ]:
import pandas as pd
df=pd.read_csv('/content/heart-disease/heart_disease_uci.csv')

In [ ]:
df.head()

In [ ]:
print(df.columns)

In [ ]:
df.isnull().sum()

In [ ]:
numeric_cols= df.select_dtypes(include='number').columns
df[numeric_cols]= df[numeric_cols].fillna(df[numeric_cols].mean())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df[numeric_cols].hist(figsize=(15,10))
plt.tight_layout()
plt.show()

In [ ]:
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Numeric Feature Correlation')
plt.show()

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.tolist() # one-hot encode All 'object' categorical columns.
if 'num' in cat_cols:
  cat_cols.remove('num') #Don't incode target column.

In [ ]:
x = df.drop('num', axis=1)
y = (df['num'] > 0).astype(int) #0: no diseas, 1: disease present.

In [ ]:
x = pd.get_dummies(x, columns=cat_cols)
print("Final festure columns:",x.columns)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled= scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression #its about classification.

In [ ]:
lr_model= LogisticRegression() #giving new data
lr_model.fit(x_train_scaled, y_train) #training step

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
y_pred_lr= lr_model.predict(x_test_scaled)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm=confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (Logistic Regression)')
plt.show

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_scaled,y_train)
y_pred_rf = rf_model.predict(x_test_scaled)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

In [ ]:
feat_imp = pd.Series(rf_model.feature_importances_, index=x.columns)
feat_imp.nlargest(10).plot(kind='barh')
plt.title('Random Forest Feature Importance')
plt.show()

In [ ]:
import joblib
joblib.dump(rf_model, 'heart_rf_model.pkl')

In [ ]:
joblib.dump(scaler, 'heart_scaler.pkl')

In [ ]:
sample= x.head(1)
sample.to_csv('Heart_user_template.csv', index=False)
print("User template saved as 'Heart_user_template.csv' ")

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import joblib
import pandas as pd

user_df = pd.read_csv('heart_dataset.csv')

#Getting columns list from traing dataframe
numeric_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols=df.select_dtypes(include='object').columns.tolist()
bool_cols=df.select_dtypes(include='bool').columns.tolist()

#Droppping columns which are extra in user_df than required to avoid error
numeric_cols=[col for col in numeric_cols if col in user_df.columns]
cat_cols=[col for col in cat_cols if col in user_df.columns]
bool_cols=[col for col in bool_cols if col in user_df.columns]

#Fill the missing numeric columns, cat columns and boolean columns
user_df[numeric_cols]= user_df[numeric_cols].fillna(user_df[numeric_cols].mean())

for col in cat_cols:
  user_df[col]=user_df[col].fillna('Unknown')

for col in bool_cols:
    user_df[col]=user_df[col].astype(int)

#One-hot encoding cat columns
user_df_encoded=pd.get_dummies(user_df, columns=cat_cols)

#Allign columns
user_df_encoded = user_df_encoded.reindex(columns=x.columns, fill_value=0)

#Scale data
scaler = joblib.load('heart_scaler.pkl')
user_df_scaled=scaler.transform(user_df_encoded)

#Prediction
model=joblib.load('heart_rf_model.pkl')
preds=model.predict(user_df_scaled)
user_df['Heart_Disease_Prediction']=preds

#Show result
print(user_df)